In [182]:
# !pip install --upgrade pip
# !pip install bs4 selenium black pandas --quiet

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from datetime import datetime

In [6]:
sites = {
    "yahoo": {
        "base_url": "https://finance.yahoo.com/topic/",
        "category": {
            "economics": "economic-news",
            "tech": "tech",
            "housing": "housing-market",
        },
    }
}

In [31]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0",
    "Referer": "",
}

In [28]:
req = requests.get("https://www.cnbc.com/industrials/")
soup = BeautifulSoup(req.content, 'html.parser')
titles = soup.find_all(class_="Card-titleContainer")
title_text = [t.find(class_="Card-title").text for t in titles  if t.find(class_="InvestingClubPill-investingClubPillLink") is None]
links = [t.find(class_="Card-title")["href"] for t in titles if t.findPrevious(class_="InvestingClubPill-investingClubPillLink") is None]

In [27]:
title_text

['JetBlue, British Airways seek partnership to expand networks',
 'Boeing shareholders reelect departing CEO Calhoun to board',
 'Frontier Airlines does away with change fees in budget airline pricing overhaul ',
 'United says FAA cleared it to start adding aircraft, routes after safety review',
 'Norfolk Southern activist Ancora wins three board seats but falls short of ousting CEO',
 'Boeing forced to scrub first crewed Starliner launch to the space station',
 'Proxy advisor ISS says unions and pension funds should back Norfolk Southern activist investor',
 "Jim Cramer's takes on 5 stocks:  Coca-Cola, McDonald's, 3M, PayPal and Walmart",
 "Glass Lewis endorses 6 of activist Ancora's nominees for Norfolk Southern board",
 'American cuts some international flights into 2025, cites Boeing delivery delays',
 'Elliott takes $1B stake in Anglo American as miner faces takeover interest',
 'Open seating no more? Southwest CEO says airline is weighing cabin changes',
 'American Airlines swing

In [17]:
titles

[<a class="Card-title" href="https://www.cnbc.com/2024/05/23/stocks-making-the-biggest-moves-before-the-bell-nvda-lyv-snow.html" target="">Stocks making the biggest moves before the bell: Nvidia, Live Nation,  Snowflake and more</a>,
 <a class="Card-title" href="https://www.cnbc.com/2024/05/22/chinas-magnificent-seven-gets-new-etf.html" target="">This ETF aims to capture China's own 'Magnificent Seven'</a>,
 <a class="Card-title" href="https://www.cnbc.com/2024/05/22/stocks-making-the-biggest-moves-after-hours-nvda-snow-vfc.html" target="">Stocks making biggest moves after hours: Nvidia, Snowflake, VF Corp. and more</a>,
 <a class="Card-title" href="https://www.cnbc.com/2024/05/22/3-big-risks-to-the-stock-market-economist-says.html" target="">These are the 3 big risks to the stock market, economist says</a>,
 <a class="Card-title" href="https://www.cnbc.com/2024/05/22/synapse-bankruptcy-customer-funds.html" target="">Fintech nightmare: 'I have nearly $38,000 tied up' after Synapse bank

In [ ]:
def parse_articles_bs(link):
    req = requests.get(link)
    soup = BeautifulSoup(req.content, 'html.parser')    
    title_text = [t.text for t in soup.find_all(class_="Mb(5px)")]
    links = [t.find('a')["href"] for t in soup.find_all(class_="Mb(5px)")]
    articles = [get_article(l) for l in links]
    
    df = pd.DataFrame({'title': title_text, 'link': links, 'content': articles})
    return df

In [178]:
def parse_articles_selenium(article_list):
    titles = article_list.find_elements(by=By.CLASS_NAME, value="sa_text")
    title_text = [t.find_element(by=By.CLASS_NAME, value="sa_text_title").text for t in titles]
    links = [t.find_element(by=By.CLASS_NAME, value="sa_text_title").get_attribute("href") for t in titles]
    articles = [get_article(l) for l in links]
    
    df = pd.DataFrame({'title': title_text, 'link': links, 'content': articles})
    return df

In [15]:
def get_article(link: str):
    req = requests.get(link)
    soup = BeautifulSoup(req.content, 'html.parser')
    date = soup.select_one("#caas-art-9dffa534-bcd1-3e24-8377-ba1fb9d5347e > article > div > div > div > div > div > div > div:nth-child(1) > div.caas-content-byline-wrapper > div.caas-attr > div > div.caas-attr-time-style > time")["datetime"]
    content = '\n'.join([p.text for p in soup.find(class_="caas-body").select("p")])
    return {"date": date, "content": content}

In [201]:
req = requests.get("https://news.naver.com/breakingnews/section/101/259")
soup = BeautifulSoup(req.content, 'html.parser')
titles = [t.text.strip('\n') for t in soup.find_all(class_="sa_text_title")]
links = [t['href'] for t in soup.find_all(class_="sa_text_title")]

In [210]:
datetime.date()

datetime.datetime(2024, 5, 17, 15, 59, 24, 340080)

In [32]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0",
    "Referer": "",
}

## Free Article

In [70]:
link = "https://www.cnbc.com/2024/05/17/hsbc-shares-fall-amid-reports-top-shareholder-ping-an-may-sell-shares.html"
req = requests.get(link, headers=request_headers)
soup = BeautifulSoup(req.content, 'html.parser')
# date = soup.select_one("time")["datetime"]
# content = '\n'.join([p.text for p in soup.find(class_="caas-body").select("p")])

In [71]:
date = soup.find("time")
date["datetime"]

'2024-05-17T04:59:24+0000'

In [72]:
[g.text for g in soup.find_all(class_="group")]


['Citing people familiar with the matter, Bloomberg said that "one option an internal team at the Chinese insurance giant is considering is further share sales, similar to the $50 million sale it disclosed last week."Ping An has butted heads with HSBC\'s management in recent years, most notably supporting a shareholder motion in 2023 that sought to spin off its Asia business and establish fixed dividends.',
 "Shares of HSBC Holdings fell over 3% in Hong Kong on Friday after reports that its top shareholder Ping An Insurance might be looking to cut its stake in the British bank.Despite the fall, HSBC's share price is still at its highest since August 2018, trading at about 68 Hong Kong dollars per share.",
 'Citing people familiar with the matter, Bloomberg reported the Chinese insurer is looking at possibly reducing its stake in the bank further "as it seeks to reduce its $13.3 billion position in Europe\'s largest lender."There are several options including "further share sales, simil

## Premium Article

In [40]:
link = "https://www.cnbc.com/2024/05/13/why-honeywell-investors-should-stay-long-despite-the-stocks-recovery.html"
req = requests.get(link, headers=request_headers)
soup = BeautifulSoup(req.content, 'html.parser')
# date = soup.select_one("time")["datetime"]
# content = '\n'.join([p.text for p in soup.find(class_="caas-body").select("p")])

In [43]:
soup.find(class_="xyz-data").text

'Every weekday the CNBC Investing Club with Jim Cramer holds a "Morning Meeting" livestream at 10:20 a.m. ET. Here\'s a recap of Monday\'s key moments. Stocks were subdued to start the week as Wall Street looks to keep its winning ways going. Last week, the Dow Jones Industrial Average secured its best weekly performance of 2024, up 2.16%, while the S & P 500 and Nasdaq advanced 1.85% and 1.14%, respectively. There was largely an "absence of news" last week, Jim Cramer noted, and yet stocks were still able to advance. "That\'s a bull market," Jim said. This week, though, inflation data will take center stage, with the release of the closely watched consumer price index Wednesday morning. Wholesale inflation data is out Tuesday. Apple shares jumped more than 1.5% Monday after Bloomberg News reported the company is nearing a deal with ChatGPT creator OpenAI to put its generative artificial intelligence technology in the next iPhone operating system. Apple declined Bloomberg\'s request fo

In [44]:
soup.find("time")

<time data-testid="published-timestamp" datetime="2024-05-13T16:14:26+0000" itemprop="datePublished">Published Mon, May 13 2024<span class="ArticleHeader-datetimeDivider"></span>12:14 PM EDT</time>

In [59]:
card_container = "Card-titleContainer"
premium_content = "InvestingClubPill-investingClubPillLink"
title_content = "Card-title"
date_selector = "time"
content_class_free = "caas-body"
content_class_premium = "xyz-data"
        
def get_article(link: str):
    print(link)
    try:
        req = requests.get(link, headers=request_headers)
        soup = BeautifulSoup(req.content, "html.parser")
        date = soup.find("time")["datetime"]
        content = (
            "\n".join(
                [g.text for g in soup.find_all(class_=content_class_free)]
            )
            if soup.find(class_=content_class_premium) is None
            else soup.find(class_=content_class_premium).text
       )
        print(content)

    except Exception as e:
        print(str(e) + link)
        return None
    return {"date": date, "content": content}


In [60]:
get_article("https://www.cnbc.com/2024/05/17/hsbc-shares-fall-amid-reports-top-shareholder-ping-an-may-sell-shares.html")

https://www.cnbc.com/2024/05/17/hsbc-shares-fall-amid-reports-top-shareholder-ping-an-may-sell-shares.html



{'date': '2024-05-17T04:59:24+0000', 'content': ''}

In [65]:
link = "https://www.cnbc.com/2024/05/17/hsbc-shares-fall-amid-reports-top-shareholder-ping-an-may-sell-shares.html"
req = requests.get(link, headers=request_headers)
content = (soup.find(class_=content_class_premium).text
        )

In [66]:
content

'Every weekday the CNBC Investing Club with Jim Cramer holds a "Morning Meeting" livestream at 10:20 a.m. ET. Here\'s a recap of Monday\'s key moments. Stocks were subdued to start the week as Wall Street looks to keep its winning ways going. Last week, the Dow Jones Industrial Average secured its best weekly performance of 2024, up 2.16%, while the S & P 500 and Nasdaq advanced 1.85% and 1.14%, respectively. There was largely an "absence of news" last week, Jim Cramer noted, and yet stocks were still able to advance. "That\'s a bull market," Jim said. This week, though, inflation data will take center stage, with the release of the closely watched consumer price index Wednesday morning. Wholesale inflation data is out Tuesday. Apple shares jumped more than 1.5% Monday after Bloomberg News reported the company is nearing a deal with ChatGPT creator OpenAI to put its generative artificial intelligence technology in the next iPhone operating system. Apple declined Bloomberg\'s request fo